In [31]:
#Loading packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import chess

from keras.models import load_model


In [2]:
df_full = pd.read_pickle('cleaned_data.pkl')
df = df_full[['board', 'encoded_board', 'move', 'encoded_move']]

In [3]:
df_shape = df['encoded_board'][0].shape
#Additional data cleaning
df = df[df['encoded_move']!=""]


x_max = np.max(df['encoded_board'].apply(lambda x: np.max(x)))
x_min = np.min(df['encoded_board'].apply(lambda x: np.min(x)))
y_max = np.max(df['encoded_move'])

print("Encoded board has a minimum of " + str(x_min) + " and a maximum of " + str(x_max))
print("There are " + str(y_max) + " different encoded moves")



Encoded board has a minimum of 0 and a maximum of 1
There are 4660 different encoded moves


In [4]:
x = df['encoded_board']
y = df['encoded_move']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

x_train = x_train.to_numpy()
x_val = x_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()


In [5]:
# x_train.reshape(6672,8,8,14)

for i in range(len(x_train)):
    x_train[i] = x_train[0].reshape(896)
    
x_train = np.array(x_train.tolist()).astype('float32')

for i in range(len(x_val)):
    x_val[i] = x_val[0].reshape(896)
    
x_val = np.array(x_val.tolist()).astype('float32')

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')

In [6]:

len(np.unique(y))
print(np.min(y))
print(np.max(y))

x_val[0].shape
y_train.shape

0
4660


(66437,)

In [7]:
input_shape = x_train[0].shape[0]
y_max = 4672 #Number of possible moves


model = keras.Sequential([
    keras.layers.Flatten(input_shape=(896,)),
    keras.layers.Dense(units=20, activation='relu'),
    keras.layers.Dense(units=y_max, activation='softmax')
])

model.compile(
    optimizer='Adam',
    loss='SparseCategoricalCrossentropy',
    metrics=['accuracy']
)

model.fit(
    x=x_train,
    y=y_train,
    epochs=4
)

Epoch 1/4
2077/2077 [==============================] - 12s 5ms/step - loss: 6.8761 - accuracy: 0.0073
Epoch 2/4
2077/2077 [==============================] - 11s 5ms/step - loss: 6.7484 - accuracy: 0.0083
Epoch 3/4
2077/2077 [==============================] - 11s 5ms/step - loss: 6.7439 - accuracy: 0.0080
Epoch 4/4
2077/2077 [==============================] - 11s 5ms/step - loss: 6.7421 - accuracy: 0.0080


In [32]:
model.save('saved_models/testing_model.h5')

In [30]:
#Reshape to be able to input to model
prediction_input = x_val[0].reshape(1, 896)
prediction = model(prediction_input)
prediction = prediction.numpy()
prediction = prediction.reshape(4672,)




0.0016174312